In [1]:
import time, os
from pymystem3 import Mystem
import re, json, requests, numpy as np
from yaml import safe_load
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster.hierarchy import linkage, fcluster
from datetime import datetime

In [2]:
def string_negative_keywords(txt_file):

    '''
    Получает название файла в рабочем каталоге
    Возвращает строку минус слов
    '''

    f = open(txt_file, 'r')#, encoding='utf-8')
    list_neg = set([re.sub(r'[\[\]"\ufeff]', '', line.strip()) for line in f])
    list_of_negative = []
    for keyword in list_neg:
        print(keyword)
        if len(keyword.split()) > 1:
            list_of_negative.append('-({})'.format(keyword))

        else:
            list_of_negative.append('-{}'.format(keyword))
    f.close()
    print(' '.join(list_of_negative))
    print(len(' '.join(list_of_negative)))
    return ' '.join(list_of_negative)


def txt_to_lists_for_wordstat_reports(txt_file):
    '''
    На вход подается текстовый файл с масками запросов. Каждая маска на отдельной строке
    Возвращает список списков масок по 10 масок - ограничение Yandex Direct API
    Списки отсортированы по убыванию количества слов в запросе
    Если в файле более 1000 масок - они отсекаются
    '''
    f = open(txt_file, 'r')
    temp_keyword_list = []
    new_list = []
    main_iterator = 1
    temp_iterator = 1

    for line in f:
        keyword_base = re.sub(r'[\[\]"+\ufeff]', '',line.strip())
        keyword_base = re.sub(r'[-,./\\#@%&^"\']', ' ', keyword_base)
        temp_keyword_list.append(keyword_base)
        temp_keyword_list.sort(key=lambda x: len(x.split()), reverse=False)
        if len(temp_keyword_list) == 1000:
            break
    f.close()
    basic_keyword_list = []
    for keyword in temp_keyword_list:
        print(keyword)
        new_list.append(keyword)
        if main_iterator == 10:  # На отладке берем 3 запроса
            basic_keyword_list.append(new_list)
            new_list = []
            temp_iterator += 1
            main_iterator = 1

            continue
        if temp_iterator == len(temp_keyword_list):
            basic_keyword_list.append(new_list)
            new_list = []
            temp_iterator = 0

        main_iterator += 1
        temp_iterator += 1

    return basic_keyword_list


def createNewWordstatReport(basic_keyword_list, negative_kw, yandex_token, api_url, proxy, geo='' ):
    '''
    Формируем новый отчет Yandex Wordstat
    '''

    basic_keywords = []
    for keyword in basic_keyword_list:
        new_word = '{} {}'.format(keyword, negative_kw)
        basic_keywords.append(new_word)

    geo = list(geo)

    data = {
        # указываем метод вордстат, с которым работаем, в данном случае - создаем отчет
        'method': 'CreateNewWordstatReport',
        # задаем входные параметры; у каждого метода они свои
        'token': yandex_token,
        'param': {
            # Указываем не более 10 фраз через запятую
            'Phrases': basic_keywords,
            # Указываем нужный регион, если не указали - Все регионы
            'GeoID': geo
        }}
    encoded_data = json.dumps(data, ensure_ascii=False).encode('utf-8')
    print(encoded_data)
    # создаем отчет
    requests.post(api_url, data=encoded_data, proxies=proxy)  # Если возвращает ошибку добавить аргумент proxies=proxy)


def getWordstatReportIdList(yandex_token, api_url, proxy):
    '''
    Возвращает список отчетов
    Структура списка [{'ReportID': 1873637, 'StatusReport': 'Done'}]
    '''
    report_data = {
        'token': yandex_token,
        'method': 'GetWordstatReportList'
    }
    report_list_encoded_data = json.dumps(report_data,
                                          ensure_ascii=False).encode('utf-8')

    return requests.post(api_url,
                         data=report_list_encoded_data, proxies=proxy).json()['data']


def deleteWordstatReport(yandex_token, api_url, report_id, proxy):
    '''
    Удаляет готовый отчет
    '''
    report_for_delete_data = {
        "method": "DeleteWordstatReport",
        "param": report_id,
        "token": yandex_token
    }
    encoded_data = json.dumps(report_for_delete_data,
                              ensure_ascii=False).encode('utf-8')
    r = requests.post(api_url, data=encoded_data, proxies=proxy)
    return r.json()['data']


def getWordstatReport(yandex_token, api_url, report_id, proxy):
    '''
    Функция возвращает отчет
    '''
    report_params = {
        'method': 'GetWordstatReport',
        'param': report_id,
        'token': yandex_token
    }
    report_encoded_data = json.dumps(report_params, ensure_ascii=False).encode('utf-8')
    # получаем отчет
    return requests.post(api_url, data=report_encoded_data, proxies=proxy).json()


def wordstatReadyReport(yandex_token, api_url, proxy):
    '''
    Получаем словарь с запросами
    '''
    time.sleep(5)
    reportList = getWordstatReportIdList(yandex_token, api_url, proxy)
    print(reportList)
    main_wordstat_report = {'data': []}
    for report_ in reportList:
        if report_["StatusReport"] == 'Done':
            # получаем отчет
            wordstat_report = getWordstatReport(yandex_token, api_url, report_["ReportID"], proxy)
            # удаляем отчет в Яндекс
            deleteWordstatReport(yandex_token, api_url, report_["ReportID"], proxy)
        elif report_["StatusReport"] == 'Failed':
            # удаляем отчет
            deleteWordstatReport(yandex_token, api_url, report_["ReportID"], proxy)
            continue
        else:
            time.sleep(5)
            if report_["StatusReport"] == 'Done':
                # запрашиваем статус отчета
                wordstat_report = getWordstatReport(yandex_token, api_url, report_["ReportID"], proxy)
                deleteWordstatReport(yandex_token, api_url, report_["ReportID"], proxy)
            else:
                deleteWordstatReport(yandex_token, api_url, report_["ReportID"], proxy)
        try:
            main_wordstat_report['data'].extend(wordstat_report['data'])
        except (KeyError, UnboundLocalError):
            continue
    return main_wordstat_report


def report_descriptor(reportDict):
    '''
    Функция лемматизирует ключевые слова и возвращает словарь с ключом = базовому слову
    Значения каждого слова словаря - список словарей структуры "ключевое слово": частота
    '''

    m = Mystem()
    keyword_dict = {}
    KW_list = []
    lemma_without_negative_word_regex = re.compile(r'^([A-ZА-Яа-яa-z\s\d+]*)\s-')
    for lemma_report in reportDict['data']:
        Started = True
        for report_dict in lemma_report['SearchedWith']:
            try:
                if Started:
                    Started = False
                    lemma_test = report_dict['Phrase']

                    lemma_search = lemma_without_negative_word_regex.search(lemma_test)
                    lemma_word = lemma_search.group(1)
                    print(lemma_word)
                    keyword_dict.setdefault(re.sub(r'\+', '', lemma_word), [])
                    continue
                word = report_dict['Phrase'].lower()
                word_show = report_dict['Shows']
                word = re.sub(r'\+', '', word)
                print(word)
                lemmas = m.lemmatize(word)
                # удаляем предлоги из леммы
                '''for part_of_speech in m.analyze(word):
                    # может быть обращение к словарю только включающему text
                    try:
                        if part_of_speech['analysis'] != []:
                            if part_of_speech['analysis'][0]['gr'] == 'PR=':
                                lemmas.remove(lemmas[lemmas.index(part_of_speech['text'])])
                    except KeyError:
                        continue'''

                for i in ['и', ' ', ',', '.', '\n', '+']:
                    while i in lemmas:
                        lemmas.remove(i)
                lemmas.sort()
                # приводим странные леммы
                if 'ипать' in lemmas:
                    lemmas[lemmas.index('ипать')] = 'ип'
                if 'банка' in lemmas:
                    lemmas[lemmas.index('банка')] = 'банк'
                if 'усна' in lemmas:
                    lemmas[lemmas.index('усна')] = 'усн'
                if 'юла' in lemmas:
                    lemmas[lemmas.index('юла')] = 'юл'

                # получаем готовое условие показа
                new_word = ' '.join(lemmas)
                if new_word in KW_list:
                    continue
                KW_list.append(new_word)
                new_word_dict = {new_word: word_show}
                keyword_dict[lemma_word].append(new_word_dict)
                if keyword_dict[lemma_word] == []:
                    keyword_dict.pop(lemma_word)
            except AttributeError:
                print('AttributeError')
    return keyword_dict, KW_list

In [7]:
API_URL = 'https://api-sandbox.direct.yandex.ru/live/v4/json/'

config_file = open('config_new.yam', mode = 'r', encoding = 'utf-8')
config = safe_load(config_file)
config_file.close()
token = config['access_token_ya_direkt']
proxy = {'http': config['proxy1'],
         'https': config['proxy2']}
# Минус слова
negativeKeywords = 'negative_kw.txt'
# Создаем строку минус слов
negative = string_negative_keywords(negativeKeywords)
# Маски запросов для отчетов
mask_txt = 'keywords.txt'
masks_lists = txt_to_lists_for_wordstat_reports(mask_txt)

# цикл по вложенным спискам
Started = True
for masks_list in masks_lists:
    createNewWordstatReport(masks_list, negative, token, API_URL, proxy, geo='43')
    if Started:
        keyword_dict = wordstatReadyReport(token, API_URL, proxy)
        Started = False
        continue
    keyword_dict['data'].extend(wordstatReadyReport(token, API_URL, proxy)['data'])

kw_Dict1, kw_list1 = report_descriptor(keyword_dict)

водитель
челябинск
эскорт
уренгой
ежедневная
монтажные
удалить
выполнение
нефтехим
на !час
!время
зеленодольск
стриптиз
массажер
место
удаленная
бывшая
год
класс
сварщик
суть
детская
уфсин
образцы
дипломный
оэз
учитель
горничная
смоленск
ремонт
математик
хостес
такси
екатеринбург
сосать
смысл
банкомат
воронеж
доу
челны
адрес
трахать
инженер
история
иваново
курсовая
строительство
диплом
график
нижнекамск
воспитатель
грузчик
выполним
охрана
москва
опыт
!часы
реферат
строитель
библиотека
принцип
контрольная
физик
бригада
мвд
удаленка
иркутск
порно
цена
массаж
стройка
вахта
скачать
уф
мерлен
член
режим
секс
алабуга
школа
охранник
бригадир
характер
-водитель -челябинск -эскорт -уренгой -ежедневная -монтажные -удалить -выполнение -нефтехим -(на !час) -!время -зеленодольск -стриптиз -массажер -место -удаленная -бывшая -год -класс -сварщик -суть -детская -уфсин -образцы -дипломный -оэз -учитель -горничная -смоленск -ремонт -математик -хостес -такси -екатеринбург -сосать -смысл -банкомат -ворон

[{'ReportID': 4111557, 'StatusReport': 'Done'}]
b'{"method": "CreateNewWordstatReport", "token": "AgAAAAAMjgdhAAXbmZLxvjkodU8GkEG4CzOGssA", "param": {"Phrases": ["\xd1\x80\xd0\xb0\xd0\xb1\xd0\xbe\xd1\x82\xd0\xb0 \xd1\x81\xd0\xb5\xd0\xb3\xd0\xbe\xd0\xb4\xd0\xbd\xd1\x8f \xd0\xba\xd0\xb0\xd0\xb7\xd0\xb0\xd0\xbd\xd1\x8c -\xd0\xb2\xd0\xbe\xd0\xb4\xd0\xb8\xd1\x82\xd0\xb5\xd0\xbb\xd1\x8c -\xd1\x87\xd0\xb5\xd0\xbb\xd1\x8f\xd0\xb1\xd0\xb8\xd0\xbd\xd1\x81\xd0\xba -\xd1\x8d\xd1\x81\xd0\xba\xd0\xbe\xd1\x80\xd1\x82 -\xd1\x83\xd1\x80\xd0\xb5\xd0\xbd\xd0\xb3\xd0\xbe\xd0\xb9 -\xd0\xb5\xd0\xb6\xd0\xb5\xd0\xb4\xd0\xbd\xd0\xb5\xd0\xb2\xd0\xbd\xd0\xb0\xd1\x8f -\xd0\xbc\xd0\xbe\xd0\xbd\xd1\x82\xd0\xb0\xd0\xb6\xd0\xbd\xd1\x8b\xd0\xb5 -\xd1\x83\xd0\xb4\xd0\xb0\xd0\xbb\xd0\xb8\xd1\x82\xd1\x8c -\xd0\xb2\xd1\x8b\xd0\xbf\xd0\xbe\xd0\xbb\xd0\xbd\xd0\xb5\xd0\xbd\xd0\xb8\xd0\xb5 -\xd0\xbd\xd0\xb5\xd1\x84\xd1\x82\xd0\xb5\xd1\x85\xd0\xb8\xd0\xbc -(\xd0\xbd\xd0\xb0 !\xd1\x87\xd0\xb0\xd1\x81) -!\xd0\xb2\xd1\x80\xd0\xb5

[{'ReportID': 4111559, 'StatusReport': 'Done'}]
вакансии казань
вакансии казань
казань работа вакансии
свежие вакансии казань
вакансии работодателя казань
вакансии казань от прямых
вакансии казань от прямых работодателей казань
работа в казани свежие вакансии
работа казань вакансии от прямых
работа казань вакансии от прямых работодателей
свежие вакансии работодателей казань
свежие вакансии казань от прямых
работа казань свежие вакансии от прямых
работа казани от прямых работодателей свежие вакансии
свежие вакансии в казани от прямых работодателей
авито казань вакансии
сайт вакансии казани
казань официальный сайт вакансии
заводы казани вакансии
авито работа казань вакансии
hh казань вакансии
бухгалтер вакансии казань
склад вакансии казань
казань ооо вакансии
ру вакансии казани
вакансии казань без
вайлдберриз казань вакансии
вакансии мужчинам казань
вакансия г казани
менеджер вакансии казань
банки казани вакансии
вакансии банков казани
хедхантер казань вакансии
вакансии дороги казань
ржд

работа медсестрой в казани свежие вакансии
авиационный завод казань вакансии
нн ру работа казань вакансии
техстрой казань вакансии
казань работа психологом вакансии
сиделка казань вакансии с проживанием
транспортная безопасность казань вакансии
аромат казань вакансии
һһ ру вакансии казань
отели казани вакансии
вакансия медсестра казань свежие
хх казань вакансии от прямых работодателей
хх ру работа казань свежие вакансии
вакансии геолога казань
вакансии машинист экскаватора казань
работа ру казань вакансии для мужчин
управление образования казань официальный сайт вакансии
лаборант казань вакансии
вакансии электрик казань
вакансия ветеринар казань
хенде казань вакансии
вакансии казань авиастроительный
вакансии казань цех
трудоустройство казань
трудоустройство казань
официальное трудоустройство казань
курсы трудоустройством казань
обучение с трудоустройства казань
медосмотр для трудоустройства казань
работа в казани трудоустройство
работа в казани официальное трудоустройство
трудоустройст

работа кондуктором в казани
работа в макдональдс казань
работа форекс казань
работа карусель казань
магазин пятерочка работа казани
сварочные работы казань
работа кфу казань
частный работа казань
работы вакансии казань погрузчики
работа на день казань
работа казань окна
работа в казани видео
работа для девушек в казани
работа дальнобойщиком в казани
работа поликлиник казань
работа магнит косметик казань
работа на дому казань вакансии
казань работа вакансии студентам
заказать работу казань
работа в казани с проживанием для женщин
прайс на отделочные работы в казани
работа на грузовых в казани
работа химиком казань
работа транспортных компаний казань
работа на грузовом автомобиле казань
монолитные работы казань
работа максидом казань
хх казань работа вакансии
работа в казани после армии
работа монтажником в казани
иннополис казань работа
работа медсестрой в казани вакансии
вакансии казань найди работу
казань работа вакансии
казань работа вакансии
работа в казани свежие вакансии
работа ка

работа в казани свежие вакансии для женщин
свежие вакансии женщинам казань
свежие вакансии в казани для мужчин
свежие вакансии на авито казань
авито казань свежие вакансии от прямых
работа в казани свежие вакансии для мужчин
авито свежие вакансии от работодателя казань
казань авито свежие вакансии от прямых работодателей
работа в коврове свежие вакансии казань
хедхантер казань свежие вакансии
свежие вакансии е казань
свежие вакансии в казани с проживанием
работа уборщицей свежие вакансии казань
вакансия уборщица казань свежие
работа медсестрой в казани свежие вакансии
вакансия медсестра казань свежие
хх ру работа казань свежие вакансии
хх ру казань свежие вакансии
работа бухгалтера в казани свежие вакансии
свежие вакансии бухгалтера казань
хх казань вакансии свежие
бухгалтер вакансии казань от прямых работодателей свежие
работа в казани на авито свежие вакансии
хедхантер казань вакансии от прямых работодателей свежие
разнорабочий казань свежие вакансии
найти работу в казани свежие вака

хх казань вакансии от прямых работодателей
вакансии казань от прямых работодателей для женщин
хх казань вакансии от прямых
хх работа казань вакансии от прямых
бухгалтер вакансии казань от прямых работодателей свежие
хедхантер казань вакансии от прямых работодателей
хедхантер казань вакансии от прямых
хедхантер казань вакансии от прямых работодателей свежие
вакансии няни в казани от прямых работодателей
медсестра вакансии казань от прямых
мерчендайзер казань вакансии от прямых
валберис казань вакансии от прямых работодателей
вакансии уборщицы казань от прямых работодателей
авито вакансии в казани от прямых
нн казань вакансии от прямых работодателей
вакансии медсестры в казани от прямых работодателей
мерчендайзер вакансии в казани от прямых работодателей
вакансии казань с проживанием от прямых работодателей
вакансии казань с проживанием от прямых
вакансии психолога в казани от прямых работодателей
вакансии в казани производство от прямых работодателей
администратор вакансии казань прямой

In [8]:
kw_Dict1

{'вакансии казань': [{'вакансия казань': 3317},
  {'вакансия казань работа': 568},
  {'вакансия казань свежий': 319},
  {'вакансия казань работодатель': 293},
  {'вакансия казань от прямой': 283},
  {'вакансия казань казань от прямой работодатель': 279},
  {'в вакансия казань работа свежий': 232},
  {'вакансия казань от прямой работа': 185},
  {'вакансия казань от прямой работа работодатель': 184},
  {'вакансия казань работодатель свежий': 177},
  {'вакансия казань от прямой свежий': 171},
  {'вакансия казань от прямой работа свежий': 153},
  {'вакансия казань от прямой работа работодатель свежий': 152},
  {'в вакансия казань от прямой работодатель свежий': 147},
  {'авито вакансия казань': 145},
  {'вакансия казань сайт': 88},
  {'вакансия казань официальный сайт': 85},
  {'вакансия завод казань': 64},
  {'авито вакансия казань работа': 50},
  {'hh вакансия казань': 44},
  {'бухгалтер вакансия казань': 36},
  {'вакансия казань склад': 35},
  {'вакансия казань ооо': 35},
  {'вакансия к

In [9]:
def kw_to_cluster(dictionary, keyword):
    '''
    Преобразование вложенных в отчет по ключевому слову словарей в список ключевых запросов
    '''
    phrases_list = []
    if len(dictionary) == 0:
        phrases_list.append(keyword)
        return phrases_list
    else:
        for dict_ in dictionary[keyword]:
            for key_kw in dict_.keys():
                phrases_list.append(key_kw)
    return phrases_list

def group_name(cluster_name, word_list):
    '''
    Возвращает название группы в виде строки
    '''
    m = Mystem()
    lemmas = m.lemmatize(cluster_name)
    for i in [' ', '.', ',', '\n']:
        while i in lemmas:
            lemmas.remove(i)
    if 'ипать' in lemmas:
        lemmas[lemmas.index('ипать')] = 'ип'
    if 'банка' in lemmas:
        lemmas[lemmas.index('банка')] = 'банк'
    if 'усна' in lemmas:
        lemmas[lemmas.index('усна')] = 'усн'
    if 'юла' in lemmas:
        lemmas[lemmas.index('юла')] = 'юл'
    # Лемма названия ключевого слова
    word_list.sort(key=lambda x: len(x.split()))
    short_word_list = word_list[0].split()

    for word in short_word_list:
        if word not in lemmas:
            gr_name_list = cluster_name.split()
            gr_name_list.append(word)
            return re.sub(r'[+]','',' '.join(gr_name_list))
    return re.sub(r'[+]','',cluster_name)


def yandex_keyword_dict(dictionary_keywords):
    '''
    Возвращает словарь групп ключевых слов с названиями
    :param dictionary_keywords: dict
    :return: dict
    '''

    new_dict = {}
    for key in dictionary_keywords.keys():
        list_exact = ['"{}"'.format(x) for x in dictionary_keywords[key]]
        new_dict[key] = [dictionary_keywords[key], list_exact]
        key_exact = '"{}"'.format(key)
        new_dict.update({key_exact: list_exact})

    return new_dict


def clusters(dictionary, freq=2):
    '''
    Кластеризация запросов с регулируемой частотой вхождения
    '''
    ready_dict = {}
    keyw_dict = {}
    for key in dictionary.keys():
        ready_dict.setdefault(key, [])
        phrases = kw_to_cluster(dictionary, key)
        if len(phrases) == 1:
            dct = {0: phrases}
        elif len(phrases) == 0:
            continue
        else:
            cv2 = CountVectorizer(min_df=freq)

            X = cv2.fit_transform(phrases)
            link_M = linkage(X.toarray(), method='ward')
            clusters = fcluster(link_M,
                                int(np.round(np.divide(len(phrases), 5))),
                                criterion='maxclust')
            dct = {}
            for key_, label in zip(phrases, clusters):
                dct[label] = dct.get(label, []) + [key_]

        for x in dct.keys():
            count = 0
            for keyword in dct[x]:
                for dict_kw in dictionary[key]:
                    if keyword in dict_kw.keys():
                        count += dict_kw[keyword]
            grName = group_name(key, dct[x])
            ready_dict[key].append([dct[x], count, grName])
            keyw_dict.setdefault(grName, dct[x])
    new_dict_kw = yandex_keyword_dict(keyw_dict)
    return ready_dict, new_dict_kw

In [10]:
phrases, kw_s = clusters(kw_Dict1)

In [11]:
phrases

{'вакансии казань': [[['вакансия казань',
    'бухгалтер вакансия казань',
    'вакансия казань ооо',
    'вакансия г казань',
    'вакансия казань менеджер',
    'банк вакансия казань',
    'вакансия дорога казань',
    'вакансия казань помощник',
    'вакансия директор казань',
    'вакансия казань руководитель',
    'вакансия казань технолог',
    'вакансия город казань',
    'администратор вакансия казань',
    'вакансия казань телематика',
    'вакансия валберис казань',
    'вакансия казань продавец',
    'вакансия казань техник',
    'аэропорт вакансия казань',
    'вакансия казань мастер',
    'вакансия казань татнефть',
    'вакансия казань чоп',
    'вакансия казань монтажник',
    'вакансия казань юрист',
    'вакансия казань мадоу',
    'вакансия казань цветок',
    'вакансия казань производство',
    'вакансия казань пятерочка',
    'вакансия казань программист',
    'вакансия казань туризм',
    'вакансия казань продажа',
    'вакансия казань озон',
    'вакансия казань р